# Description

This notebook analyzes overlap between:
- the list of Human genes, targeted by SARS-COV-2 viral miRNAs (as predicted by our pipeline) and
- the list of Human genes, downregulated in people infected with SARS-COV-2 (as discovered in gene expression datasets).

We consider two sets of target genes:
- with stricter filtering (score > 95 AND number of targets per miRNA < 800)
- with laxer filtering (score > 95)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Load predicted genes

In [2]:
targets_basedir = Path('../targets/non-conserved-region')

mirnafold_path = targets_basedir / 'mirnafold-pipeline/mirnafold-targets/score-more-95-targets-less-800.csv'
mirnafold_path_lax = targets_basedir / 'mirnafold-pipeline/mirnafold-targets/score-more-95.csv'
vmir_path = targets_basedir / 'vmir-pipeline/vmir-targets/score-more-95-targets-less-800.csv'
vmir_path_lax = targets_basedir / 'vmir-pipeline/vmir-targets/score-more-95.csv'

In [3]:
mirnafold_targets = pd.read_csv(mirnafold_path)
mirnafold_targets_lax = pd.read_csv(mirnafold_path_lax)
vmir_targets = pd.read_csv(vmir_path)
vmir_targets_lax = pd.read_csv(vmir_path_lax)

print('mirnafold:',len(mirnafold_targets), 'vmir:', len(vmir_targets))
print('mirnafold (laxer filtering):',len(mirnafold_targets_lax), 'vmir laxer filtering:', len(vmir_targets_lax))

mirnafold: 50 vmir: 44
mirnafold (laxer filtering): 399 vmir laxer filtering: 102


In [4]:
# merge target predictions from both pipelines.
# in case of duplicates, keep one with highest score (confidence)
predicted_genes_df = pd.merge(mirnafold_targets, vmir_targets, how='outer')
predicted_genes_df = predicted_genes_df.sort_values('Target Score', ascending=False).drop_duplicates('Gene Symbol', keep='first') 
predicted_genes_df

,Target Rank,Target Score,Sequence,Gene Symbol,Gene Description,Strand,# of predicted targets,Seed
67,2,100,AUUGCCAUAGUAAUGGUGACAA,ZNF493,zinc finger protein 493,5',653,UUGCCAU
66,1,100,AUUGCCAUAGUAAUGGUGACAA,ZNF99,zinc finger protein 99,5',653,UUGCCAU
21,1,100,AUAGUGUUUAUAACACUUUGCU,MAP4K3,mitogen-activated protein kinase kinase kinase...,5',656,UAGUGUU
84,3,99,ACAAUUAUGCUUUGCUGUAUGA,TMEM56,transmembrane protein 56,3',667,CAAUUAU
63,1,99,UGAUUCUCUUCCUGUUCCAAGC,FRK,fyn related Src family tyrosine kinase,3',476,GAUUCUC
...,...,...,...,...,...,...,...,...
58,9,96,UCAGCAACACAGUUGCUGAUUC,SLK,STE20 like kinase,5',762,CAGCAAC
59,10,96,UCAGCAACACAGUUGCUGAUUC,LARP4B,La ribonucleoprotein domain family member 4B,5',762,CAGCAAC
60,11,96,UCAGCAACACAGUUGCUGAUUC,IL4R,interleukin 4 receptor,5',762,CAGCAAC
61,12,96,UCAGCAACACAGUUGCUGAUUC,TMEM178B,transmembrane protein 178B,5',762,CAGCAAC


In [5]:
# merge target predictions from both pipelines.
# in case of duplicates, keep one with highest score (confidence)
predicted_genes_lax_df = pd.merge(mirnafold_targets_lax, vmir_targets_lax, how='outer')
predicted_genes_lax_df = predicted_genes_lax_df.sort_values('Target Score', ascending=False).drop_duplicates('Gene Symbol', keep='first') 
predicted_genes_lax_df

,Target Rank,Target Score,Sequence,Gene Symbol,Gene Description
0,1,100,AUUCUAAUUUCUCCACGUCUUU,ICA1L,islet cell autoantigen 1 like
110,14,100,AUUCAUUUGUAAUUAGAGGUGA,TEAD1,TEA domain transcription factor 1
108,12,100,AUUCAUUUGUAAUUAGAGGUGA,VCAN,versican
107,11,100,AUUCAUUUGUAAUUAGAGGUGA,RPRD1B,regulation of nuclear pre-mRNA domain containi...
106,10,100,AUUCAUUUGUAAUUAGAGGUGA,OXR1,oxidation resistance 1
...,...,...,...,...,...
294,198,96,AUUCAUUUGUAAUUAGAGGUGA,RAB22A,"RAB22A, member RAS oncogene family"
295,199,96,AUUCAUUUGUAAUUAGAGGUGA,ADAMTS6,ADAM metallopeptidase with thrombospondin type...
296,200,96,AUUCAUUUGUAAUUAGAGGUGA,DCUN1D5,defective in cullin neddylation 1 domain conta...
297,201,96,AUUCAUUUGUAAUUAGAGGUGA,TMEM233,transmembrane protein 233


# Load differentially expressed genes (DEGs)

These are some significant DEGs in patients with COVID from supplementary materials at: https://academic.oup.com/cid/article/71/16/2052/5822600#supplementary-data

- The reported DEGs are all **down**-regulated
- They come from two conditions: severe and mild

In [16]:
!pip install python-docx openpyxl # xlrd 

     |████████████████████████████████| 243 kB 2.0 MB/s eta 0:00:01


In [6]:
from docx.api import Document

# adopted from https://stackoverflow.com/a/27862205/7948839
def parse_table(path_to_doc, table_id):
    document = Document(path_to_doc)
    
    if table_id > len(document.tables):
        raise f'There are only {len(document.tables)} tables in the document, but you asked for table {table_id}.'
    
    table = document.tables[table_id-1]

    rows = [] # a list of dictionaries containing each row's data.

    keys = None
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)

        # Establish the mapping based on the first row
        # headers; these will become the keys of our dictionary
        if i == 0:
            keys = tuple(text)
            continue

        # Construct a dictionary for this row, mapping
        # keys to values for this row
        row_data = dict(zip(keys, text))
        rows.append(row_data)
        
    return pd.DataFrame(rows)

In [7]:
degs_doc_path = 'ciaa462_suppl_supplymentary_materials.docx' # download from https://academic.oup.com/cid/article/71/16/2052/5822600#supplementary-data

severe_degs = parse_table(degs_doc_path, 5)
severe_degs = severe_degs.rename(columns={'A1_Label': 'Gene Symbol'})
severe_degs.drop(columns=['Degree', 'Betweenness'], inplace=True)

mild_degs = parse_table(degs_doc_path, 6)
mild_degs = mild_degs.rename(columns={'B1_Label': 'Gene Symbol'})
mild_degs.drop(columns=['Degree', 'Betweenness'], inplace=True)

degs_df = pd.merge(severe_degs, mild_degs, how='outer', on='Gene Symbol')
conditions_mapping = {'Expression_x': 'Expression_severe', 'Expression_y': 'Expression_mild'}
degs_df = degs_df.rename(columns=conditions_mapping)

# Check overlap

### Stricter miRNA filtering

In [8]:
pd.merge(predicted_genes_df, degs_df, how='inner', on='Gene Symbol')

,Target Rank,Target Score,Sequence,Gene Symbol,Gene Description,Strand,# of predicted targets,Seed,Expression_severe,Expression_mild
0,7,98,AUAGUGUUUAUAACACUUUGCU,SOS1,SOS Ras/Rac guanine nucleotide exchange factor 1,5',656,UAGUGUU,-1.85018,NaN
1,2,96,UCAUUACUUCAGGUGAUGGCAC,PIAS2,protein inhibitor of activated STAT 2,5',435,CAUUACU,-2.38271,-2.29114
2,21,96,AUAGUGUUUAUAACACUUUGCU,TNFRSF1A,TNF receptor superfamily member 1A,5',656,UAGUGUU,-1.19104,-1.69855


### Laxer miRNA filtering

In [9]:
pd.merge(predicted_genes_lax_df, degs_df, how='inner', on='Gene Symbol')

,Target Rank,Target Score,Sequence,Gene Symbol,Gene Description,Expression_severe,Expression_mild
0,31,100,AUUCAUUUGUAAUUAGAGGUGA,PTEN,phosphatase and tensin homolog,-1.29593,-1.15863
1,16,98,AUGAAGAAGGUAACAUGUUCAA,AHR,aryl hydrocarbon receptor,-1.35261,-2.21396
2,7,98,AUAGUGUUUAUAACACUUUGCU,SOS1,SOS Ras/Rac guanine nucleotide exchange factor 1,-1.85018,NaN
3,146,97,AUUCAUUUGUAAUUAGAGGUGA,PIAS2,protein inhibitor of activated STAT 2,-2.38271,-2.29114
4,21,96,AUAGUGUUUAUAACACUUUGCU,TNFRSF1A,TNF receptor superfamily member 1A,-1.19104,-1.69855


# Confusing dataset ahead (work in progress)

The dataset: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE162911

It should have 9 COVID-infected and 3 non-infected donors.

Which donors are which? I checked all files but didn't understand it. From the sheet "ROI_IDs_across_modalities" within the excel file, it looks as if there's data on COVID antigen status only for different autopsy samples within Trachea and LUL (lungs, I assume?) from the same donor (D20).

**But where is data for each individual donor in general?**

**Note:** it's probably super obvious and I'm just missing some paragraph in some file where everything is explained.

____

When we figure the question above out, we can use the dataframe below to find DEGs. 

The table already normalized expression counts, so I'll just calculate the log-fold-change values and take the ones that are significant (smaller than -1).

In [29]:
sheets = pd.read_excel('diff-expressed-sup4.xlsx', sheet_name=None)
cell_names = list(sheets.keys())
print(f'Loaded differential expression tables from {len(cell_names)} cells.')
print('\nCell names:')
print('\n- '.join(['']+cell_names))

Loaded differential expression tables from 19 cells.

Cell names:

- CD4+ T cell_linear
- AT2_linear
- club_linear
- CD8+ T cell_linear
- Treg_linear
- CD16+ monocyte_linear
- smc_linear
- monocyte_linear
- AT1_linear
- ciliated_linear
- plasma cell_linear
- goblet_linear
- lymphatic endothelial_linear
- pericyte_linear
- vascular endothelial_linear
- NK cell_linear
- mast_linear
- macrophage_linear
- B cell_linear


In [41]:
dict(zip(cell_names, [None]*(len(cell_names))))

{'CD4+ T cell_linear': None,
 'AT2_linear': None,
 'club_linear': None,
 'CD8+ T cell_linear': None,
 'Treg_linear': None,
 'CD16+ monocyte_linear': None,
 'smc_linear': None,
 'monocyte_linear': None,
 'AT1_linear': None,
 'ciliated_linear': None,
 'plasma cell_linear': None,
 'goblet_linear': None,
 'lymphatic endothelial_linear': None,
 'pericyte_linear': None,
 'vascular endothelial_linear': None,
 'NK cell_linear': None,
 'mast_linear': None,
 'macrophage_linear': None,
 'B cell_linear': None}

In [94]:
cell_type_ddegs = dict(zip(cell_names, [None]*(len(cell_names))))

# merged_df = pd.DataFrame(columns=['Gene'])
merged = set()

for cell_type in cell_names:
    df = sheets[cell_type]
    
    df = df.rename(columns={'Unnamed: 0': 'Gene'})
    
    # select significant DEGs
    df = df[df.loc[:, 'sig'] == True]
    
    # select downregulated DEGs
    df = df[df.loc[:, 'disease_log2fc'] < 0]
    
    # save cell type
    df.loc[:, 'cell_type'] = cell_type
    
    # merge
    ddeg_set = set(list(df.loc[:, 'Gene']))
    merged = merged.union(ddeg_set)
    
    cell_type_ddegs[cell_type] = ddeg_set
    
#     merged_df = pd.merge(merged_df, df, how='right', on='Gene')
#     merged_df = pd.merge(merged_df, df[cols_to_use], on='Gene')
    
#     display(df)
#     break
    
#     cell_type_ddegs[cell_type] = df
    
    print(f'Found {len(df)} downregulated genes in this cell type')

print(f'Total genes: {len(merged)}')

Found 114 downregulated genes in this cell type
Found 197 downregulated genes in this cell type
Found 187 downregulated genes in this cell type
Found 80 downregulated genes in this cell type
Found 152 downregulated genes in this cell type
Found 104 downregulated genes in this cell type
Found 144 downregulated genes in this cell type
Found 201 downregulated genes in this cell type
Found 135 downregulated genes in this cell type
Found 315 downregulated genes in this cell type
Found 489 downregulated genes in this cell type
Found 662 downregulated genes in this cell type
Found 128 downregulated genes in this cell type
Found 567 downregulated genes in this cell type
Found 120 downregulated genes in this cell type
Found 138 downregulated genes in this cell type
Found 195 downregulated genes in this cell type
Found 201 downregulated genes in this cell type
Found 161 downregulated genes in this cell type
Total genes: 1889


In [102]:
predicted_lax_set = set(list(predicted_genes_lax_df.loc[:, 'Gene Symbol']))
overlap = predicted_lax_set.intersection(merged)
print(f'{len(overlap)} out of {len(predicted_genes_lax_df)} predicted genes that are empirically down-regulated in COVID-infected patients.')

31 out of 472 predicted genes that are empirically down-regulated in COVID-infected patients.


In [110]:
from collections import defaultdict
d = defaultdict(list)
for k, v in cell_type_ddegs_overlap.items():
    d[tuple(v)].append(list(k))

In [125]:
column_names = list(cell_type_ddegs.keys())

In [142]:
cell_type_ddegs_overlap = {cell_type:genes.intersection(predicted_lax_set) for cell_type, genes in cell_type_ddegs.items()}

In [143]:
gene_to_cells = {}
for cell_type, genes in cell_type_ddegs_overlap.items():
    for gene in genes:
        if gene not in gene_to_cells:
            gene_to_cells[gene] = ['-'] * len(column_names)
        
        
        gene_to_cells[gene][column_names.index(cell_type)] = 'Down-regulated'

results = pd.DataFrame(gene_to_cells).T
results.columns = column_names

print('Down-regulated genes, by cell type:')
results
    

Down-regulated genes, by cell type:


,CD4+ T cell_linear,AT2_linear,club_linear,CD8+ T cell_linear,Treg_linear,CD16+ monocyte_linear,smc_linear,monocyte_linear,AT1_linear,ciliated_linear,plasma cell_linear,goblet_linear,lymphatic endothelial_linear,pericyte_linear,vascular endothelial_linear,NK cell_linear,mast_linear,macrophage_linear,B cell_linear
GABPB1,Down-regulated,-,-,Down-regulated,Down-regulated,-,-,Down-regulated,-,-,-,-,-,Down-regulated,-,Down-regulated,Down-regulated,-,Down-regulated
SRSF2,Down-regulated,-,-,Down-regulated,Down-regulated,-,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-
MYLIP,Down-regulated,-,-,Down-regulated,Down-regulated,-,-,Down-regulated,-,-,-,-,-,-,-,Down-regulated,Down-regulated,-,Down-regulated
ZFP36,-,Down-regulated,-,Down-regulated,Down-regulated,-,Down-regulated,-,-,-,Down-regulated,-,Down-regulated,Down-regulated,Down-regulated,Down-regulated,Down-regulated,Down-regulated,Down-regulated
NECAB1,-,Down-regulated,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
ZFAND5,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-,Down-regulated,-,-,-,Down-regulated,-
IRS2,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-,Down-regulated,-,-,-,-,-
HSBP1,-,-,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-,-,-,-,-
AQP4,-,-,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-,-,-,-,-
FAM92B,-,-,-,-,-,-,-,-,-,Down-regulated,-,-,-,-,-,-,-,-,-
